In [65]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [9]:
# run search
result = client.search("What is the AQI in New delhi?",
                       include_answer=True)

In [10]:
# print the answer
result['results'][0]

{'title': 'New Delhi Air Quality Index (AQI) : Real-Time Air Pollution',
 'url': 'https://www.aqi.in/dashboard/india/delhi/new-delhi',
 'content': 'Current New Delhi Air Quality Index (AQI) is 273 Unhealthy level with real-time air pollution PM2.5 (114µg/m³), PM10 (229µg/m³), Temperature (19.5°C) in Delhi.',
 'score': 0.9466806,
 'raw_content': None}

### Using an agent with persistence

In [56]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

In [78]:
tavily_search_tool = TavilySearchResults(max_results=2)

In [15]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [57]:
class Agent:
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        # self.memory = SqliteSaver.from_conn_string(":memory:")
        self.memory = MemorySaver()
        self.graph = graph.compile(checkpointer=self.memory)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [79]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tavily_search_tool], system=prompt)

In [34]:
messages = [HumanMessage(content="What is the weather in new delhi?")]
thread = {"configurable": {"thread_id": "1"}}

In [59]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JZl7PoeGS0ynQoWgGteIWaCD', 'function': {'arguments': '{"query":"current weather in New Delhi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9ab6b7ce-c1e3-4944-983b-d923ddb60be6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in New Delhi'}, 'id': 'call_JZl7PoeGS0ynQoWgGteIWaCD', 'type': 'tool_call'}], usage_metadata={'input_tokens': 153, 'output_tokens': 23, 'total_tokens': 176, 'input_token_details': {'audi

In [60]:
messages = [HumanMessage(content="What about in ghaziabad?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9KcgCNj0Pfi5XqihsTKYuP44', 'function': {'arguments': '{"query":"current weather in Ghaziabad"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 956, 'total_tokens': 980, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-838183c8-1bc9-4c26-a7bf-51e92ecb76a4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Ghaziabad'}, 'id': 'call_9KcgCNj0Pfi5XqihsTKYuP44', 'type': 'tool_call'}], usage_metadata={'input_tokens': 956, 'output_tokens': 24, 'total_tokens': 980, 'input_token_det

In [61]:
messages = [HumanMessage(content="which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Both New Delhi and Ghaziabad currently have the same temperature of 22.2°C (72.0°F). Therefore, neither is warmer than the other at this moment.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 1617, 'total_tokens': 1655, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1536}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'stop', 'logprobs': None}, id='run-36dab3e2-b585-49ed-8150-3365c013bbe8-0', usage_metadata={'input_tokens': 1617, 'output_tokens': 38, 'total_tokens': 1655, 'input_token_details': {'audio': 0, 'cache_read': 1536}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [62]:
messages = [HumanMessage(content="Which of them have less AQI?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oSQn3kiqLuTBXxUB1IZoAWtM', 'function': {'arguments': '{"query": "current AQI in New Delhi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_iXOlYXV3UfSNibrsSVIAluDB', 'function': {'arguments': '{"query": "current AQI in Ghaziabad"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 1669, 'total_tokens': 1733, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1536}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-075f921b-f022-4525-9122-1f1ffbf04148-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current AQI in New D

### some more custom tools to enhance ability of the agents

In [73]:
import requests

class CityAQITool:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "http://api.openweathermap.org/data/2.5/air_pollution"

    def get_city_aqi(self, city_name):
        # Fetch city coordinates
        geo_url = f"http://api.openweathermap.org/geo/1.0/direct"
        geo_params = {
            "q": city_name,
            "appid": self.api_key
        }
        geo_response = requests.get(geo_url, params=geo_params)
        if geo_response.status_code != 200:
            return f"Error fetching coordinates: {geo_response.status_code} - {geo_response.text}"

        geo_data = geo_response.json()
        if not geo_data:
            return f"City '{city_name}' not found."

        # Get latitude and longitude
        lat = geo_data[0]['lat']
        lon = geo_data[0]['lon']

        # Fetch AQI data
        aqi_params = {
            "lat": lat,
            "lon": lon,
            "appid": self.api_key
        }
        aqi_response = requests.get(self.base_url, params=aqi_params)
        if aqi_response.status_code != 200:
            return f"Error fetching AQI data: {aqi_response.status_code} - {aqi_response.text}"

        aqi_data = aqi_response.json()
        print(aqi_data)
        aqi = aqi_data['list'][0]['main']['aqi']
        pm2_5 =  aqi_data['list'][0]['components']['pm2_5']

        # AQI Mapping
        aqi_levels = {
            1: "Good",
            2: "Fair",
            3: "Moderate",
            4: "Poor",
            5: "Very Poor"
        }

        return f"The AQI for {city_name} is {aqi} ({aqi_levels.get(aqi, 'Unknown')} level). The PM 2.5 is {pm2_5} ug/m3"


In [74]:
# Example usage:
# Replace YOUR_API_KEY with your OpenWeatherMap API key
aqi_tool = CityAQITool(api_key=os.environ.get("WEATHER_API_KEY"))
result = aqi_tool.get_city_aqi("New delhi")
print(result)

{'coord': {'lon': 77.209, 'lat': 28.6139}, 'list': [{'main': {'aqi': 5}, 'components': {'co': 2403.26, 'no': 0.75, 'no2': 86.37, 'o3': 35.41, 'so2': 26.46, 'pm2_5': 224.38, 'pm10': 294.54, 'nh3': 23.81}, 'dt': 1735126246}]}
The AQI for New delhi is 5 (Very Poor level). The PM 2.5 is 224.38 ug/m3


In [81]:
from langchain.tools import tool

@tool
def get_city_aqi(city_name: str) -> str:
    """
    Fetches the live AQI of a city by name.
    Args:
    - city_name (str): The name of the city.
    
    Returns:
    - str: AQI and its description.
    """
    return aqi_tool.get_city_aqi(city_name)

In [85]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tavily_search_tool,get_city_aqi], system=prompt)

In [87]:
messages = [HumanMessage(content="compare AQIs of new delhi and ghaziabad")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6U1MYbHNtDN3i6kIXXKLoRBu', 'function': {'arguments': '{"city_name": "New Delhi"}', 'name': 'get_city_aqi'}, 'type': 'function'}, {'id': 'call_iRchhquAE3NszYbLbxwOoEJm', 'function': {'arguments': '{"city_name": "Ghaziabad"}', 'name': 'get_city_aqi'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 245, 'total_tokens': 299, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2eb2eed5-a56a-4bcf-a93e-1590b68e1068-0', tool_calls=[{'name': 'get_city_aqi', 'args': {'city_name': 'New Delhi'}, 'id': 'call_6U1MYbHNtDN3i6kIXXKLoRBu', 'type': 'tool_call'}, {'name': 'g